# CS 503 lecture Notes(1/23/25) 
## Digging into Syscalls
- HW due this sunday is the same as last weeks but have to manually manage string data


- Syscall
    - they are an architecture-specific way for your program to ask the OS to do something priviledged

    - Think of the OS as a gatekeeper to priviledged operations that only the kernel can do
    -implicit vs explicit syscalls:
        - many syscalls are required to run any executable, even if it doesn't make any calls of its own(implicit)
            - ex. your shell and OS have to make syscalls to load, run, and exit a program
            - gcc include libc startup/teardown code
            - you can explicitly invoke syscalls to operate in kernelspace from your program
    - When looking into and tracking explicit calls use the syscall() function from libc
        - ex. pidt pid = syscall(SYS_getpid); where pid = process id
- how do I know what all the syscalls available on my system are?
    - you can run the "ausyscall --dump" audit syscall function to return all syscalls available to you in the given context
        - running this will give you ~460 syscalls and their associated syscall id ex. 172 = getpid (get process id)
        - if you run: ausyscall --dump | grep getpid  => 172 getpid
    - you can also look at C header files
- man7.org has linux documentation for syscalls

### switching from user to kernel space ("context swithcing") occurs every syscall
- the userspace of your CPU contains the "state of your program including stack & registers" -->> and it gets saved to a "process-specific kernel stack" -hydrates state->> to of kernel stack and makes syscall executions neccessary against the CPU in "kernelspace" 
    - and this process is reversed in order to refresh the state of the userspace after the syscall!
- Context switching is "Expensive"! in terms of time 1,000,000 context switches is equivalent to 0.8seconds
    - only use syscalls when required, use libraires that avoid syscalls, batch syscalls where possible ("buffers")
- passing args to syscalls from C
    - they may take simple types like a "long"
    - other times you may have to pass C structs/ that deal with "struct alignement"



### A LOOK AT POINTERS
- a pointer is a value calculated in an expression. Its an integer that acts as a memory address.
    - where char a[10] is an array of char. 
    where:
        a[0] = memory address: 0xe014
        a[1] = 0xe015
        a[2] = 0xe016
        a[6] = 0xe01a
        a[9] = 0xe01d
    - **An array name evaluates to be a pointer to the first element**
    - so getting the address of a == &(a[0]), where address of set a is equivalent to the memory location of the first index of the array.
        - &() (ampersands) gives the address of the element within its parentheses
    - In that same respect a[0] == \*a where: (* = the contents of or known as the dereferencing operator) *of and address is the contents of that address
    - a[i] == *(a+i) for example if a[0]== 0xe014 then a[3]==0xe017
    - in a[2]; the a is the pointer to the first memory location of the array and the 2 moves along 2 memory slots.
    - pointers are the way we calculate where something in memory is.
    - the only thing you do with pointers is:
        - one: add pointers and integers
        - two: subtract pointers from eachother
    - Int b[10] is an int array
        where: 
        Int *p; in a pointer to that array
        b[0] = 0x1010
        b[1] = 0x1014 (we jumped 4 slots since a int is 32 bits which is 4 bytes) (divide by8)
        b[2] = 0x1018
        b[3] = 0x101c
        b[4] = 0x1020
        b[10] = 0x1038
        for (i=0,p=b;i<10; i++, p++)        where ox100c:[0x1010]P  - where ox100c is the memory location of the pointer P that points to the location of the first element of the array
            \*P*2                       this line *P takes the contents(value) of this memory location or b[0] and multiply it by 2.
                                        P is in units of integer so adding 1(iterating) to P is really adding 4 to the pointer so now it is pointing to the value at b[1] which is then multiplied by 2. whereas incrementing i is just adding 1. so we iterate through pointing to each array value and multipling it till we reach the end of the array where i is no longer less than 10
        *p is the contents of address slot of p. b = memory location b[0].
        






### A LOOK AT STRUCTS(half-way through video)
- A struct (short for structure): is a user-defined data type that allows you to group different types of variables inder a single name. These variables known as members or fields can be of different types and are grouped together for easier and mor logical organization. 
    - Structs allow us to represent complex data by bundling multiple variables of potentially different types into one cohesive unit
    - basic syntax:
        //define a struct to represent a "point" in 2D space
        struct Point{
            int x;
            int y;
        };
        int main(){
            //declare and initialize a struct variable
            struct Point p1 = {10, 20};
            
            //use pointers with structs
            struct Point *ptr = &p1; // Pointer to struct Point


            // accessing the memebers of the struct
            printf("point p1: (%d, %d)\n", p1.x, p1.y);

            //accessing the struct members via the pointer
            printf("Point p1:(%d, %d)\n", ptr->x, ptr->y)

            return 0;
        }



### A LOOK at MALLOC AND VALGRIND
- your code binary(lowaddress) -> literal/const -> global/static -> HEAP(HEAP on exists in c if we dynamically allocate mememory for the program) -> STACK(high address)
    - HEAP is managed by libc functions(which call syscalls) - used whenever we need dynamic/variable memory amounts
    - STACK is managed by compiler generated code(deterministic)
- Typically you want to program using stack allocation because the code for memory management is handled by the compiler
- Using heap allocation memory is neccessary if your dealing with dynamic data where you don't know the size of an inputed variable for example until the runtime so you need to use a buffer to handle unknown size of a string. 
- Buffers are created using malloc()
- Valgrind 




